# Web Scraping

In [1]:
from bs4 import BeautifulSoup
import requests 
import pandas as pd 

In [2]:
url = "https://www.usbanklocations.com/bank-rank/total-assets.html?d=2021-03-31"

response = requests.get(url)

In [3]:
response.status_code

200

In [4]:
soup=BeautifulSoup(response.content, 'html.parser')

In [51]:
table = soup.select("#left > table")[0].text


In [53]:
table = soup.select("#left > table")[0]

In [54]:
table

<table><tr valign="bottom"><td width="48"><b>Rank</b></td><td width="200"><b>Total Assets</b></td><td><b>Bank Name</b></td></tr><tr><td>1</td><td>$3,207,521,000,000</td><td><a href="/jpmorgan-chase-bank-locations.htm">JPMorgan Chase Bank</a></td></tr>
<tr><td>2</td><td>$2,316,773,000,000</td><td><a href="/bank-of-america-locations.htm">Bank of America</a></td></tr>
<tr><td>3</td><td>$1,773,157,000,000</td><td><a href="/wells-fargo-bank-locations.htm">Wells Fargo Bank</a></td></tr>
<tr><td>4</td><td>$1,684,634,000,000</td><td><a href="/citibank-locations.htm">Citibank</a></td></tr>
<tr><td>5</td><td>$542,566,283,000</td><td><a href="/u-s-bank-locations.htm">U.S. Bank</a></td></tr>
<tr><td>6</td><td>$506,471,000,000</td><td><a href="/truist-bank-locations.htm">Truist Bank</a></td></tr>
<tr><td>7</td><td>$469,299,477,000</td><td><a href="/pnc-bank-locations.htm">PNC Bank</a></td></tr>
<tr><td>8</td><td>$411,622,119,000</td><td><a href="/td-bank-locations.htm">TD Bank</a></td></tr>
<tr><td

In [55]:
table = soup.select("#left > table")[0]

In [59]:
import pandas as pd
table_rows = table.find_all('tr')


l = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text for tr in td]
    l.append(row)

bank_rank =pd.DataFrame(l, columns=["Rank", "Total Assets", "Bank Name"])

In [60]:
df['Total Assets'][1:].apply(lambda x: int(x.lstrip('$').replace(",","")))

1       3207521000000
2       2316773000000
3       1773157000000
4       1684634000000
5        542566283000
            ...      
4983          7996000
4984          7990000
4985          4095000
4986          3698000
4987          3408000
Name: Total Assets, Length: 4987, dtype: int64

In [62]:
bank_rank.head()

,Rank,Total Assets,Bank Name
0,Rank,Total Assets,Bank Name
1,1,"$3,207,521,000,000",JPMorgan Chase Bank
2,2,"$2,316,773,000,000",Bank of America
3,3,"$1,773,157,000,000",Wells Fargo Bank
4,4,"$1,684,634,000,000",Citibank


In [66]:
bank_rank=bank_rank.drop(index=0)

In [15]:
bank_rank.head(50)

NameError: name 'bank_rank' is not defined

In [68]:
bank_rank.to_csv("bank_rank.csv")

# Unifying Bank Name with Company column 

In [1]:
import pandas as pd

In [2]:
complaints_data=pd.read_csv('complaint_data.csv')

In [128]:
complaints_data.shape

(1471766, 18)

In [3]:
bank_rank=pd.read_csv('bank_rank.csv')

In [4]:
bank_rank.head()

,Unnamed: 0,Rank,Total Assets,Bank Name
0,1,1,"$3,207,521,000,000",JPMorgan Chase Bank
1,2,2,"$2,316,773,000,000",Bank of America
2,3,3,"$1,773,157,000,000",Wells Fargo Bank
3,4,4,"$1,684,634,000,000",Citibank
4,5,5,"$542,566,283,000",U.S. Bank


In [9]:
bank_rank["Bank Name"].str.lower()


0                                     jpmorgan chase bank
1                                         bank of america
2                                        wells fargo bank
3                                                citibank
4                                               u.s. bank
                              ...                        
4982                                   bmo harris central
4983    interstate federal savings and loan associatio...
4984                                   dsrm national bank
4985        kentland federal savings and loan association
4986                             emigrant mercantile bank
Name: Bank Name, Length: 4987, dtype: object

In [39]:
complaints_data["Company"].str.lower()

0          experian information solutions inc.
1                                    aes/pheaa
2                                equifax, inc.
3                        wells fargo & company
4                                discover bank
                          ...                 
1471761        american financial network inc.
1471762          diversified consultants, inc.
1471763            ocwen financial corporation
1471764               freedom mortgage company
1471765                           nelnet, inc.
Name: Company, Length: 1471766, dtype: object

In [40]:
bank_rank['Bank Name'] = bank_rank['Bank Name'].str.rstrip('bank')

In [41]:
bank_rank.head()

,Unnamed: 0,Rank,Total Assets,Bank Name
0,1,1,"$3,207,521,000,000",JPMorgan Chase B
1,2,2,"$2,316,773,000,000",Bank of Americ
2,3,3,"$1,773,157,000,000",Wells Fargo B
3,4,4,"$1,684,634,000,000",Citi
4,5,5,"$542,566,283,000",U.S. B


In [42]:
import random
from difflib import SequenceMatcher
banks = complaints_data['Company'].tolist()
banks_ = bank_rank['Bank Name'].tolist()

df1 = pd.DataFrame({'Company':[random.choice(banks) for i in range(300)]}, index=range(300))
df2 = pd.DataFrame({'Company':[random.choice(banks_) for i in range(300)]}, index=range(300))


def similar(a, b):
    # returns 1 when full match, 0 if no character matches.
    return SequenceMatcher(None, a, b).ratio()

In [43]:
unique_banks = df2['Company'].unique()

In [44]:
def similar(bank2test, unique_banks_list):
    # returns match
    dct = {bank_:SequenceMatcher(None, bank2test, bank_).ratio() for bank_ in unique_banks_list}
    return max(dct, key=dct.get)

In [45]:
similar('JPMORGAN CHASE', unique_banks)

'MRV Banks'

In [46]:
unique_banks = df1['Company'].unique()

In [56]:
unique_banks

array(['Experian Information Solutions Inc.',
       'Ocwen Financial Corporation', 'WELLS FARGO & COMPANY',
       'BANK OF AMERICA, NATIONAL ASSOCIATION', 'M&T BANK CORPORATION',
       'Coinbase, Inc.', 'JPMORGAN CHASE & CO.',
       'TD BANK US HOLDING COMPANY', 'AMERICAN HONDA FINANCE CORP',
       'SANTANDER BANK, NATIONAL ASSOCIATION',
       'TRANSUNION INTERMEDIATE HOLDINGS, INC.', 'U.S. BANCORP',
       "Dyck-O'Neal, Inc.", 'TRANSWORLD SYSTEMS INC',
       'WESTERN UNION COMPANY, THE', 'Suburban Credit Corporation',
       'Sentry Recovery and Collections, Inc.',
       'Dovenmuehle Mortgage, Inc.', 'Ditech Financial LLC',
       'UNITED SERVICES AUTOMOBILE ASSOCIATION', 'CITIBANK, N.A.',
       'Credit Bureau Collection Services, Inc.', '21ST MORTGAGE CORP.',
       'EQUIFAX, INC.', 'SYNCHRONY FINANCIAL', 'SUNTRUST BANKS, INC.',
       'DriveTime', 'CARRINGTON MORTGAGE SERVICES, LLC',
       'CCS Financial Services, Inc.', 'STERLING JEWELERS, INC.',
       'GOLDMAN SACHS BAN

In [47]:
df2['company_cleaned'] = df2['Company'].apply(lambda x: similar(x, unique_banks))

In [48]:
df2[['company_cleaned', 'Company']]

,company_cleaned,Company
0,PNC Bank N.A.,Bank of Chi
1,"Stanislaus Credit Control Service, Inc.",First Central Bank McCoo
2,"Halsted Financial Services, LLC.",Valley Bank of Kalispell
3,"Convergent Resources, Inc.",Evergreen Bank Group
4,Synergetic Communication Inc,"Peoples Community Bank SB of Monticello, Indi"
...,...,...
295,"Southwest Credit Systems, L.P.",Simmesport State B
296,"Duncan Solutions, lnc.",Exchange State B
297,"Fair Resolutions, Inc.",First National B
298,"General Motors Financial Company, Inc.",Legacy Bank & Trust Company


In [49]:
complaints_data_unique=complaints_data['Company'].unique()

In [50]:
len(complaints_data_unique)

5499

In [51]:
bank_rank_unique=bank_rank['Bank Name'].unique()

In [52]:
len(bank_rank_unique)

4338

In [53]:


df1 = pd.DataFrame({'Company':[random.choice(complaints_data_unique) for i in range(100)]}, index=range(100))
df2 = pd.DataFrame({'Company':[random.choice(bank_rank_unique) for i in range(1000)]}, index=range(1000))


def similar(a, b):
    # returns 1 when full match, 0 if no character matches.
    return SequenceMatcher(None, a, b).ratio()

In [54]:
df1.head()

,Company
0,"National Account Systems of Madison, Inc"
1,TitleMax LLC
2,"United Collection Service, Inc (FL)"
3,"Bayfield Financial, LLC"
4,The CFM Group


In [55]:
df2.head()

,Company
0,Shinhan Bank Americ
1,Home Federal Savings and Loan Association of N...
2,ETHIC
3,United Bank of Unio
4,Vision B


In [56]:
unique_banks = df2['Company'].unique()

In [57]:
def similar(bank2test, unique_banks_list):
    # returns match
    dct = {bank_rank_unique:SequenceMatcher(None, bank2test, bank_rank_unique).ratio() for bank_rank_unique in unique_banks_list}
    return max(dct, key=dct.get)

In [58]:
similar('JP Morgan', unique_banks)

'South Georgia B'

In [59]:
df1['company_cleaned'] = df1['Company'].apply(lambda x: similar(x, unique_banks))

In [60]:
df1[['company_cleaned', 'Company']]

,company_cleaned,Company
0,First National Bank of Chadro,"National Account Systems of Madison, Inc"
1,Timberland B,TitleMax LLC
2,United Roosevelt Savings B,"United Collection Service, Inc (FL)"
3,ESB Financial,"Bayfield Financial, LLC"
4,Choice Financial Group,The CFM Group
...,...,...
95,First Service B,"Account Recovery Service, Inc. (WI)"
96,Metropolitan B,Meredian Inc
97,"Morgantown Bank & Trust Company, Incorporated",Maryville Collection Service Incorporated
98,"BankWest, Inc.","Jon Barry and Associates, Inc."


In [32]:
complaints_data.shape

(1471766, 18)

# Joinin two tables on Company Column

In [33]:
df = pd.merge(complaints_data, bank_rank[['Bank Name','Rank']], how= 'left', left_on= ['Company'], right_on= ['Bank Name'])

In [34]:
df.shape

(1471766, 20)

In [35]:
df.head()

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID,Bank Name,Rank
0,08/09/2015,Credit reporting,NaN,Incorrect information on credit report,Information is not mine,NaN,Company chooses not to provide a public response,Experian Information Solutions Inc.,NJ,08872,NaN,Consent not provided,Web,08/09/2015,Closed with non-monetary relief,Yes,No,1509954,NaN,NaN
1,12/23/2019,Student loan,Federal student loan servicing,Dealing with your lender or servicer,Trouble with how payments are being handled,NaN,NaN,AES/PHEAA,MA,019XX,NaN,NaN,Web,12/23/2019,Closed with explanation,Yes,NaN,3475943,NaN,NaN
2,01/29/2019,"Credit reporting, credit repair services, or o...",Credit reporting,Problem with a credit reporting company's inve...,Was not notified of investigation status or re...,NaN,NaN,"EQUIFAX, INC.",NY,10801,NaN,Consent not provided,Web,01/30/2019,Closed with non-monetary relief,Yes,NaN,3136759,NaN,NaN
3,08/19/2015,Mortgage,Conventional adjustable mortgage (ARM),"Loan servicing, payments, escrow account",NaN,NaN,Company chooses not to provide a public response,WELLS FARGO & COMPANY,CA,94526,NaN,Consent not provided,Web,08/19/2015,Closed with explanation,Yes,No,1527601,NaN,NaN
4,03/04/2016,Credit card,NaN,Billing disputes,NaN,I am dissatisfied with the current outcome of ...,NaN,DISCOVER BANK,NV,891XX,NaN,Consent provided,Web,03/04/2016,Closed with explanation,Yes,Yes,1816726,NaN,NaN


In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1471766 entries, 0 to 1471765
Data columns (total 20 columns):
 #   Column                        Non-Null Count    Dtype  
---  ------                        --------------    -----  
 0   Date received                 1471766 non-null  object 
 1   Product                       1471766 non-null  object 
 2   Sub-product                   1236601 non-null  object 
 3   Issue                         1471766 non-null  object 
 4   Sub-issue                     916763 non-null   object 
 5   Consumer complaint narrative  481087 non-null   object 
 6   Company public response       548101 non-null   object 
 7   Company                       1471766 non-null  object 
 8   State                         1447286 non-null  object 
 9   ZIP code                      1334581 non-null  object 
 10  Tags                          200565 non-null   object 
 11  Consumer consent provided?    862588 non-null   object 
 12  Submitted via               

# NLP

In [37]:
import docx2txt
import nltk 

In [38]:
from nltk.tokenize import word_tokenize
from nltk.text import Text